### Import packages

In [1]:
import pandas as pd
from cassandra.cluster import Cluster

### Create a session connection to Cassandra cluster

In [2]:
clstr = Cluster()
session = clstr.connect()

In [3]:
session.execute("DROP KEYSPACE IF EXISTS m14")

### Use session to 'talk' to cassandra

In [4]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


In [5]:
session.execute("CREATE KEYSPACE IF NOT EXISTS m14 WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':2}")

In [6]:
rows = session.execute("desc keyspaces")
for row in rows:
    print(f"{row[0]}")

m14
system
system_auth
system_distributed
system_schema
system_traces
system_views
system_virtual_schema
w04python


Create a INVENTORY table for testing

In [7]:
session.execute("""
CREATE TABLE IF NOT EXISTS m14.inventory ( 
    SKU INT,
    name Text,
    description Text,
    warehouse_num  INT,

    PRIMARY KEY(SKU,warehouse_num)
);
""")

### Load data

In [8]:
df = pd.read_csv('bddata.csv')
df

,SKU,name,description,warehouse_num
0,101,'bodyspray','controls body odour',10
1,102,'vitamin-c','supplement of c vitamin',11
2,103,'serum','helps hair growth',12
3,104,'face wash','clenses skin',13
4,105,'body wash','used to bathe',14


In [9]:
for index, row in df.iterrows():
    print(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )
    session.execute(f"""
        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES ({row[0]}, {row[1]}, {row[2]}, {row[3]});
        """
       )


        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (101, 'bodyspray', 'controls body odour', 10);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (102, 'vitamin-c', 'supplement of c vitamin', 11);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (103, 'serum', 'helps hair growth', 12);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (104, 'face wash', 'clenses skin', 13);
        

        INSERT INTO m14.inventory (SKU, name, description, warehouse_num)     
        VALUES (105, 'body wash', 'used to bathe', 14);
        


In [10]:
rows = session.execute("select (SKU, name, description, warehouse_num) from m14.inventory")
for row in rows:
    print(f"sku={row[0][0]}, name={row[0][1]}, description={row[0][2]}, warehouse_num={row[0][3]}")


sku=105, name=body wash, description=used to bathe, warehouse_num=14
sku=104, name=face wash, description=clenses skin, warehouse_num=13
sku=102, name=vitamin-c, description=supplement of c vitamin, warehouse_num=11
sku=101, name=bodyspray, description=controls body odour, warehouse_num=10
sku=103, name=serum, description=helps hair growth, warehouse_num=12


In [11]:
session.execute("CREATE INDEX warehouse_index ON m14.inventory(warehouse_num)")

In [12]:
result = session.execute("select * from m14.inventory Where warehouse_num=12 ")
for rows in result:
    print(rows)

Row(sku=103, warehouse_num=12, description='helps hair growth', name='serum')
